<a href="https://colab.research.google.com/github/Yin-Tzu/predict_DA_reaction_product/blob/main/%E7%AC%AC%E4%B8%89%E5%80%8B%E6%B7%B1%E5%BA%A6%E5%AD%B8%E7%BF%92(5.HOMO%E8%88%87LUMO%E4%B9%8BDiels-Alder%E5%8F%8D%E6%87%89%E4%BD%8D%E7%BD%AE%E6%B7%B1%E5%BA%A6%E5%AD%B8%E7%BF%92%E9%A0%90%E6%B8%AC)/3rdNN_dataset2_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.linalg as la
from PIL import Image
import copy
from sklearn.model_selection import train_test_split
import random
 
print(np.append([[[1],[3]],[[5],[7]]], [[[9],[11]],[[13],[15]]], axis=2))  # https://blog.csdn.net/hustqb/article/details/78090365

a=np.zeros(3)
a[0]=0; a[1]=1; a[2]=2
print((a==[0,1,2]).any())	#True
print((a==[0,1,2]).all())	#True
print ((a==[1,2,3]).any())        #False
print ((a==[1,2,3]).all())        #False
print ((a==[0,2,3]).any())        #True
print ((a==[0,2,3]).all())
 

reaction_rgb10=[]
reaction_rgb15=0
reaction_rgb20=0


def x(num):  # https://www.runoob.com/python3/python3-factorial.html
  factorial = 1
  for i in range(1,num + 1):
    factorial = factorial*i
  return factorial
rgbHOMO, rgbLUMO=[],[]
c_num_H, c_num_L = [],[]
for b in [2,4,6,8,10,12,14,16,18,20]: # f
    c = int((20-b)/2)  # 矩陣旁邊加c個0
    H=np.zeros([b,b]).astype('int64')
    for j in range(b-1):
        p1 = j+1
        p2 = j
        #print(p1, p2)
        H[p1, p2] = 1
        H[p2, p1] = 1
    #print(H)
    eigvals, eigvecs = la.eig(H)
    eigvals = eigvals.real
    #eigvals = eigvals.astype('float32')
    #print(eigvals)
    #print(eigvecs)
    #fig = plt.imshow(eigvecs, cmap='binary')
    #plt.xticks(())
    #plt.yticks(())
    #plt.show()
    eigvals=eigvals.tolist()
    HOMO = eigvals.index(sorted(eigvals,reverse = True)[int((b/2)-1)])
    #print(HOMO)
    LUMO = eigvals.index(sorted(eigvals,reverse = True)[int((b/2))])
    #print(LUMO)
    g1 = 0
    for e in range(len(eigvals)):
      r, r1, r2 = [], 0, 0 
      pn =x(len(eigvals))/((x(len(eigvals)-2))*(x(2)))  # 每個軌域中選出兩個不同碳，有幾種分法
      ppnn = 0
      #print(pn,ppnn)
      if (HOMO == g1)and (len(eigvals)!=2):  # HOMO是雙烯，要四個碳
        while ppnn != pn:  # 算次數，次數沒到繼續執行
          while ([r1,r2] in r) or ([r2,r1] in r) or r1==r2: # 隨機產生r
            r1 = random.randint(0,len(eigvals)-1)
            r2 = random.randint(0,len(eigvals)-1)
          r.append([r1,r2])
          g=np.zeros(len(eigvals)).reshape(len(eigvals),1)  # 第二層:綠色
          bl = np.zeros(len(eigvals)).reshape(len(eigvals),1)  # 第三層:藍色
          re=copy.deepcopy(eigvecs[:, e].reshape(len(eigvals),1))  # 第一層:紅色
          #print(re)
          g[r1] = copy.deepcopy(re[r1])  # 紅色與綠色換數值
          g[r2] = copy.deepcopy(re[r2])  # 紅色與綠色換數值
          re[r1] = copy.deepcopy(0)  # 紅色與綠色換數值
          re[r2] = copy.deepcopy(0)  # 紅色與綠色換數值
          g=copy.deepcopy(g[:,:,np.newaxis])  # 兩維變三維
          bl=copy.deepcopy(bl[:,:,np.newaxis])
          re=copy.deepcopy(re[:,:,np.newaxis])
          gg = []
          for i in [re,g,bl]:  # 紅綠藍都變成20*1
            #print(c,'c')
            for d in range(c):
              i = np.insert(i, 0, np.zeros(1), 0)
              i = np.insert(i, np.size(i,0), np.zeros(1), 0)
            gg.append(i)
          re = copy.deepcopy(gg[0])
          g = copy.deepcopy(gg[1])
          bl = copy.deepcopy(gg[2])
          #print(r1,r2)
          #print(re)
          #print(g)
          reg = np.append(re, g, axis=2)  #　（：：在這裡合併）
          regbl = np.append(reg, bl, axis=2)
          regbl = copy.deepcopy(regbl[np.newaxis, :])
          if rgbHOMO==[]:
            rgbHOMO = copy.deepcopy(regbl)
          else:
            rgbHOMO = copy.deepcopy(np.concatenate((rgbHOMO, regbl), axis=0))
          c_num_H.append(b)
          print(rgbHOMO.shape)
          print(len(c_num_H))
          ppnn += 1
      elif LUMO == g1:
        for reverse in range(2):
          u = 0
          if reverse==1:
            
            for i in range(len(eigvecs[:, e])):
              if np.sign(eigvecs[:, e][i]) != np.sign(eigvecs[:, e][::-1][i]):
                u+=1
            if u==0:
              print(eigvecs[:, e])
              break
          print(u,reverse)
          r, r1, r2 = [], 0, 0 
          pn =x(len(eigvals))/((x(len(eigvals)-2))*(x(2)))
          ppnn = 0
          while ppnn != pn:
            while ([r1,r2] in r) or ([r2,r1] in r) or r1==r2: 
              r1 = random.randint(0,len(eigvals)-1)
              r2 = random.randint(0,len(eigvals)-1)
            r.append([r1,r2])
            g=np.zeros(len(eigvals)).reshape(len(eigvals),1)  # https://blog.csdn.net/qq_28618765/article/details/78085457
            bl = np.zeros(len(eigvals)).reshape(len(eigvals),1)
            if reverse==0:
              re=copy.deepcopy(eigvecs[:, e].reshape(len(eigvals),1))
              #print(re)
            else:
              re=copy.deepcopy(eigvecs[:, e][::-1].reshape(len(eigvals),1))
              #print(re)
            g[r1] = copy.deepcopy(re[r1])
            g[r2] = copy.deepcopy(re[r2])
            re[r1] = copy.deepcopy(0)
            re[r2] = copy.deepcopy(0)
            g=copy.deepcopy(g[:,:,np.newaxis])
            bl=copy.deepcopy(bl[:,:,np.newaxis])
            re=copy.deepcopy(re[:,:,np.newaxis])
            gg = []
            for i in [re,g,bl]:
              #print(c,'c')
              for d in range(c):
                i = np.insert(i, 0, np.zeros(1), 0)  # https://blog.csdn.net/weixin_37895339/article/details/78442555
                i = np.insert(i, np.size(i,0), np.zeros(1), 0)  # https://blog.csdn.net/qq_25436597/article/details/79079435
              gg.append(i)
            re = copy.deepcopy(gg[0])
            g = copy.deepcopy(gg[1])
            bl = copy.deepcopy(gg[2])
            #print(r1,r2)
            #print(re)
            #print(g)
            reg = np.append(re, g, axis=2)  #　（：：在這裡合併）
            regbl = np.append(reg, bl, axis=2)
            regbl = copy.deepcopy(regbl[np.newaxis, :])
            if rgbLUMO==[]:
              rgbLUMO = copy.deepcopy(regbl)
            else:
              rgbLUMO = copy.deepcopy(np.concatenate((rgbLUMO, regbl), axis=0))
            #print(regbl)
            ppnn += 1
            c_num_L.append(b)
        #print(rgbLUMO.shape)
        
      else:
        l = 0
      g1 += 1
for i in range(len(rgbHOMO)):  # 判斷label
  for j in range(len(rgbLUMO)):
    #print(i,j)
    reaction_rgb = copy.deepcopy(np.append(rgbHOMO[i], rgbLUMO[j], axis=1))  # HOMOLUMO合併
    #print(reaction_rgb.shape)
    n,m=0,0
    for k in range(20):
      #print(rgbHOMO[i][k,0,1])
      if (rgbHOMO[i][k,0,1] !=0) and m==0 :
        n+=1
        #print(n)
        m=1
        HOMOT = copy.deepcopy(rgbHOMO[i][k,0,1])
        if c_num_H[i]%4 == 0:
          if k%2 != 0:
            l=0
            #print(k%2)
            break  # HOMO最上面的反應發生位置一定要是偶數
        else:
          if k%2 != 1:
            l=0
            #print(k%2)
            break  # HOMO最上面的反應發生位置一定要是偶數
      elif (rgbHOMO[i][k,0,1] ==0) and n!=0:
        n+=1
        #print(n)
      elif (rgbHOMO[i][k,0,1] !=0) and m==1:
        if n != 3:
          l=0
          #print(rgbHOMO[i][k,0,1])
          break  # HOMO沒有隔兩格就不行
        else:
          HOMOD = copy.deepcopy(rgbHOMO[i][k,0,1])
          n1, m1 = 0,0
          for k2 in range(20):
            #print(rgbLUMO[j][k2,0,1])
            if (rgbLUMO[j][k2,0,1] !=0) and m1==0 :
              n1+=1
              #print(n)
              m1=1
              LUMOT = copy.deepcopy(rgbLUMO[j][k2,0,1])
              if c_num_L[j]%4 == 0:
                if k2%2 != 0:
                  l=0
                  #print(k%2)
                  break  # HOMO最上面的反應發生位置一定要是偶數
              else:
                if k2%2 != 1:
                  l=0
                  #print(k%2)
                  break  # HOMO最上面的反應發生位置一定要是偶數
            elif (rgbLUMO[j][k2,0,1] ==0) and n1!=0:
              n1+=1
              #print(n)
            elif (rgbLUMO[j][k2,0,1] !=0) and m1==1:
              #print(rgbHOMO[i][k,0,1])
              
              if n1 != 1:
                l=0
                break  # LUMO不能有格
              else:
                LUMOD = copy.deepcopy(rgbLUMO[j][k2,0,1])
                #print(HOMOT)
                #print(HOMOD)
                #print(LUMOT)
                #print(LUMOD)
                if (np.sign(HOMOT)==np.sign(LUMOT)) and (np.sign(HOMOD)==np.sign(LUMOD)):  # https://www.twblogs.net/a/5c4acea1bd9eee6e7d81c542
                  l=1
                  break  # 正負相同才能反映
                else:
                  l=0
                  break
            else:
              l=0
      else:
        l=0
    #print(i,j,l)
    #print(len(rgbHOMO))
    reaction_rgb = copy.deepcopy(reaction_rgb[np.newaxis, :])
    if i==0 and j==0:
      reaction_rgb50 = copy.deepcopy(reaction_rgb)
      label = copy.deepcopy(np.array([l]))
    elif i<50:
      reaction_rgb50 = copy.deepcopy(np.concatenate([reaction_rgb50, reaction_rgb]))
      label = copy.deepcopy(np.concatenate((label, np.array([l])), axis=0))
      print(reaction_rgb50.shape)
    elif i==50 and j==0:
      reaction_rgb100 = copy.deepcopy(reaction_rgb)
      label = copy.deepcopy(np.concatenate((label, np.array([l])), axis=0))
    elif i<100:
      reaction_rgb100 = copy.deepcopy(np.concatenate([reaction_rgb100, reaction_rgb]))
      label = copy.deepcopy(np.concatenate((label, np.array([l])), axis=0))
      print(reaction_rgb100.shape)
    elif i==100 and j==0:
      reaction_rgb150 = copy.deepcopy(reaction_rgb)
      label = copy.deepcopy(np.concatenate((label, np.array([l])), axis=0))
    elif i<150:
      reaction_rgb150 = copy.deepcopy(np.concatenate([reaction_rgb150, reaction_rgb]))
      label = copy.deepcopy(np.concatenate((label, np.array([l])), axis=0))
      print(reaction_rgb150.shape)
    elif i==150 and j==0:
      reaction_rgb200 = copy.deepcopy(reaction_rgb)
      label = copy.deepcopy(np.concatenate((label, np.array([l])), axis=0))
    elif i<200:
      reaction_rgb200 = copy.deepcopy(np.concatenate([reaction_rgb200, reaction_rgb]))
      label = copy.deepcopy(np.concatenate((label, np.array([l])), axis=0))
      print(reaction_rgb200.shape)
    elif i==200 and j==0:
      reaction_rgb250 = copy.deepcopy(reaction_rgb)
      label = copy.deepcopy(np.concatenate((label, np.array([l])), axis=0))
    elif i<250:
      reaction_rgb250 = copy.deepcopy(np.concatenate([reaction_rgb250, reaction_rgb]))
      label = copy.deepcopy(np.concatenate((label, np.array([l])), axis=0))
      print(reaction_rgb250.shape)
    elif i==250 and j==0:
      reaction_rgb300 = copy.deepcopy(reaction_rgb)
      label = copy.deepcopy(np.concatenate((label, np.array([l])), axis=0))
    elif i<300:
      reaction_rgb300 = copy.deepcopy(np.concatenate([reaction_rgb300, reaction_rgb]))
      label = copy.deepcopy(np.concatenate((label, np.array([l])), axis=0))
      print(reaction_rgb300.shape)
    elif i==300 and j==0:
      reaction_rgb350 = copy.deepcopy(reaction_rgb)
      label = copy.deepcopy(np.concatenate((label, np.array([l])), axis=0))
    elif i<350:
      reaction_rgb350 = copy.deepcopy(np.concatenate([reaction_rgb350, reaction_rgb]))
      label = copy.deepcopy(np.concatenate((label, np.array([l])), axis=0))
      print(reaction_rgb350.shape)
    elif i==350 and j==0:
      reaction_rgb400 = copy.deepcopy(reaction_rgb)
      label = copy.deepcopy(np.concatenate((label, np.array([l])), axis=0))
    elif i<400:
      reaction_rgb400 = copy.deepcopy(np.concatenate([reaction_rgb400, reaction_rgb]))
      label = copy.deepcopy(np.concatenate((label, np.array([l])), axis=0))
      print(reaction_rgb400.shape)
    elif i==400 and j==0:
      reaction_rgb450 = copy.deepcopy(reaction_rgb)
      label = copy.deepcopy(np.concatenate((label, np.array([l])), axis=0))
    elif i<450:
      reaction_rgb450 = copy.deepcopy(np.concatenate([reaction_rgb450, reaction_rgb]))
      label = copy.deepcopy(np.concatenate((label, np.array([l])), axis=0))
      print(reaction_rgb450.shape)
    elif i==450 and j==0:
      reaction_rgb500 = copy.deepcopy(reaction_rgb)
      label = copy.deepcopy(np.concatenate((label, np.array([l])), axis=0))
    elif i<500:
      reaction_rgb500 = copy.deepcopy(np.concatenate([reaction_rgb500, reaction_rgb]))
      label = copy.deepcopy(np.concatenate((label, np.array([l])), axis=0))
      print(reaction_rgb500.shape)
    elif i==500 and j==0:
      reaction_rgb550 = copy.deepcopy(reaction_rgb)
      label = copy.deepcopy(np.concatenate((label, np.array([l])), axis=0))
    elif i<550:
      reaction_rgb550 = copy.deepcopy(np.concatenate([reaction_rgb550, reaction_rgb]))
      label = copy.deepcopy(np.concatenate((label, np.array([l])), axis=0))
      print(reaction_rgb550.shape)
    elif i==550 and j==0:
      reaction_rgb600 = copy.deepcopy(reaction_rgb)
      label = copy.deepcopy(np.concatenate((label, np.array([l])), axis=0))
    elif i<600:
      reaction_rgb600 = copy.deepcopy(np.concatenate([reaction_rgb600, reaction_rgb]))
      label = copy.deepcopy(np.concatenate((label, np.array([l])), axis=0))
      print(reaction_rgb600.shape)
    elif i==600 and j==0:
      reaction_rgb650 = copy.deepcopy(reaction_rgb)
      label = copy.deepcopy(np.concatenate((label, np.array([l])), axis=0))
    elif i<650:
      reaction_rgb650 = copy.deepcopy(np.concatenate([reaction_rgb650, reaction_rgb]))
      label = copy.deepcopy(np.concatenate((label, np.array([l])), axis=0))
      print(reaction_rgb650.shape)
    elif i==650 and j==0:
      reaction_rgb700 = copy.deepcopy(reaction_rgb)
      label = copy.deepcopy(np.concatenate((label, np.array([l])), axis=0))
    elif i<700:
      reaction_rgb700 = copy.deepcopy(np.concatenate([reaction_rgb700, reaction_rgb]))
      label = copy.deepcopy(np.concatenate((label, np.array([l])), axis=0))
      print(reaction_rgb700.shape)
    elif i==700 and j==0:
      reaction_rgb750 = copy.deepcopy(reaction_rgb)
      label = copy.deepcopy(np.concatenate((label, np.array([l])), axis=0))
    elif i<750:
      reaction_rgb750 = copy.deepcopy(np.concatenate([reaction_rgb750, reaction_rgb]))
      label = copy.deepcopy(np.concatenate((label, np.array([l])), axis=0))
      print(reaction_rgb750.shape)
    else:
      print(i)



reaction_rgb2= copy.deepcopy(np.concatenate([reaction_rgb50, reaction_rgb100]))
reaction_rgb2= copy.deepcopy(np.concatenate([reaction_rgb2, reaction_rgb150]))
reaction_rgb2= copy.deepcopy(np.concatenate([reaction_rgb2, reaction_rgb200]))
reaction_rgb2= copy.deepcopy(np.concatenate([reaction_rgb2, reaction_rgb250]))
reaction_rgb2= copy.deepcopy(np.concatenate([reaction_rgb2, reaction_rgb300]))
reaction_rgb2= copy.deepcopy(np.concatenate([reaction_rgb2, reaction_rgb350]))
reaction_rgb2= copy.deepcopy(np.concatenate([reaction_rgb2, reaction_rgb400]))
reaction_rgb2= copy.deepcopy(np.concatenate([reaction_rgb2, reaction_rgb450]))
reaction_rgb2= copy.deepcopy(np.concatenate([reaction_rgb2, reaction_rgb500]))
reaction_rgb2= copy.deepcopy(np.concatenate([reaction_rgb2, reaction_rgb550]))
reaction_rgb2= copy.deepcopy(np.concatenate([reaction_rgb2, reaction_rgb600]))
reaction_rgb2= copy.deepcopy(np.concatenate([reaction_rgb2, reaction_rgb650]))
reaction_rgb2= copy.deepcopy(np.concatenate([reaction_rgb2, reaction_rgb700]))
reaction_rgb2= copy.deepcopy(np.concatenate([reaction_rgb2, reaction_rgb750]))
print(reaction_rgb2.shape, label.shape)
print(label)

"""  # 找到label為1的數據
ab = label.tolist()
print(ab)
find=1
for i,v in enumerate(ab):
  if v == find:
    print(i)
    print(reaction_rgb2[i:(i+1)], label[i:(i+1)])
"""
X_train, X_test, y_train, y_test = train_test_split(reaction_rgb2, label, test_size=0.2, random_state=20, shuffle=True)  # 打亂樣本資料順序
np.save('X_train', X_train)
np.save('X_test', X_test)
np.save('y_train', y_train)
np.save('y_test', y_test)

[[[ 1  9]
  [ 3 11]]

 [[ 5 13]
  [ 7 15]]]
True
True
False
False
True
False
0 0
2 1
0 0
[ 0.60150096 -0.37174803 -0.37174803  0.60150096]
(1, 20, 1, 3)
1
(2, 20, 1, 3)
2
(3, 20, 1, 3)
3
(4, 20, 1, 3)
4
(5, 20, 1, 3)
5
(6, 20, 1, 3)
6
0 0
6 1
(7, 20, 1, 3)
7
(8, 20, 1, 3)
8
(9, 20, 1, 3)
9
(10, 20, 1, 3)
10
(11, 20, 1, 3)
11
(12, 20, 1, 3)
12
(13, 20, 1, 3)
13
(14, 20, 1, 3)
14
(15, 20, 1, 3)
15
(16, 20, 1, 3)
16
(17, 20, 1, 3)
17
(18, 20, 1, 3)
18
(19, 20, 1, 3)
19
(20, 20, 1, 3)
20
(21, 20, 1, 3)
21
0 0
[ 0.46424283 -0.16122984 -0.40824829  0.30301299  0.30301299 -0.40824829
 -0.16122984  0.46424283]
(22, 20, 1, 3)
22
(23, 20, 1, 3)
23
(24, 20, 1, 3)
24
(25, 20, 1, 3)
25
(26, 20, 1, 3)
26
(27, 20, 1, 3)
27
(28, 20, 1, 3)
28
(29, 20, 1, 3)
29
(30, 20, 1, 3)
30
(31, 20, 1, 3)
31
(32, 20, 1, 3)
32
(33, 20, 1, 3)
33
(34, 20, 1, 3)
34
(35, 20, 1, 3)
35
(36, 20, 1, 3)
36
(37, 20, 1, 3)
37
(38, 20, 1, 3)
38
(39, 20, 1, 3)
39
(40, 20, 1, 3)
40
(41, 20, 1, 3)
41
(42, 20, 1, 3)
42
(43, 20, 1, 

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:155: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:96: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


串流輸出內容已截斷至最後 5000 行。
(9284, 20, 2, 3)
(9285, 20, 2, 3)
(9286, 20, 2, 3)
(9287, 20, 2, 3)
(9288, 20, 2, 3)
(9289, 20, 2, 3)
(9290, 20, 2, 3)
(9291, 20, 2, 3)
(9292, 20, 2, 3)
(9293, 20, 2, 3)
(9294, 20, 2, 3)
(9295, 20, 2, 3)
(9296, 20, 2, 3)
(9297, 20, 2, 3)
(9298, 20, 2, 3)
(9299, 20, 2, 3)
(9300, 20, 2, 3)
(9301, 20, 2, 3)
(9302, 20, 2, 3)
(9303, 20, 2, 3)
(9304, 20, 2, 3)
(9305, 20, 2, 3)
(9306, 20, 2, 3)
(9307, 20, 2, 3)
(9308, 20, 2, 3)
(9309, 20, 2, 3)
(9310, 20, 2, 3)
(9311, 20, 2, 3)
(9312, 20, 2, 3)
(9313, 20, 2, 3)
(9314, 20, 2, 3)
(9315, 20, 2, 3)
(9316, 20, 2, 3)
(9317, 20, 2, 3)
(9318, 20, 2, 3)
(9319, 20, 2, 3)
(9320, 20, 2, 3)
(9321, 20, 2, 3)
(9322, 20, 2, 3)
(9323, 20, 2, 3)
(9324, 20, 2, 3)
(9325, 20, 2, 3)
(9326, 20, 2, 3)
(9327, 20, 2, 3)
(9328, 20, 2, 3)
(9329, 20, 2, 3)
(9330, 20, 2, 3)
(9331, 20, 2, 3)
(9332, 20, 2, 3)
(9333, 20, 2, 3)
(9334, 20, 2, 3)
(9335, 20, 2, 3)
(9336, 20, 2, 3)
(9337, 20, 2, 3)
(9338, 20, 2, 3)
(9339, 20, 2, 3)
(9340, 20, 2, 3)
(9341, 20,

分割資料

In [2]:
from google.colab import drive  # https://towardsdatascience.com/importing-data-to-google-colab-the-clean-way-5ceef9e9e3c8
drive.mount('/content/drive')

Mounted at /content/drive


In [35]:
import tensorflow as tf
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import time
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler
import cv2
import copy
from sklearn.model_selection import train_test_split
import random



tStart = time.time()#計時開始
datafile1 = u'./drive/My Drive/X_train.npy'
datafile2 = u'./drive/My Drive/y_train.npy'
datafile3 = u'./drive/My Drive/X_test.npy'
datafile4 = u'./drive/My Drive/y_test.npy'
X_train_image,y_train_label,X_test_image,y_test_label = np.load(datafile1), np.load(datafile2), np.load(datafile3), np.load(datafile4)

print(X_train_image.shape,y_train_label.shape,X_test_image.shape,y_test_label.shape)

one = np.where(y_train_label == 1)
zero = np.where(y_train_label == 0)
#one = np.where(y_test_label == 1)
NDSi0 = []

#print(zero)
zero = zero[0]
#print(np.array(zero))
one = copy.deepcopy(one[0])
print(len(one))
print(len(zero))
#print(zero.dtype)
NDSi1 = []
#print(type(NDSi1))
andsit=0

p=0
j,k=0,0
for i in range(len(X_train_image)):
  if (i in zero):
    if k>0 and (len(NDSi1) == 0):
      i1=copy.deepcopy(X_train_image[p:(k+p)])
      l1=copy.deepcopy(y_train_label[p:(k+p)])
      NDSi1 = copy.deepcopy(i1)
      NDSl1 = copy.deepcopy(l1)
      k=copy.deepcopy(0)
      p=copy.deepcopy(i)  # 這裡是存1但是if進來的i是0所以p不用加1
    elif k>0 and (len(NDSi1) > 0):
      print(y_train_label[p:(k+p)],'ytrain')
      i1=copy.deepcopy(X_train_image[p:(k+p)])
      l1=copy.deepcopy(y_train_label[p:(k+p)])
      NDSi1 = copy.deepcopy(np.concatenate((NDSi1, i1), axis=0))
      NDSl1 = copy.deepcopy(np.concatenate((NDSl1, l1), axis=0))
      k=copy.deepcopy(0)
      p=copy.deepcopy(i)
      print(NDSi1.shape)
    else:
      None
    if ((len(NDSi0)+j) < 98200):
      j+=1
      #print(j)
    elif ((len(NDSi0)+j) == 98200) and j>0:
      i0=copy.deepcopy(X_train_image[p:(j+p)])
      l0=copy.deepcopy(y_train_label[p:(j+p)])
      NDSi0 = copy.deepcopy(np.concatenate((NDSi0, i0), axis=0))
      NDSl0 = copy.deepcopy(np.concatenate((NDSl0, l0), axis=0))
      j=copy.deepcopy(0)
      p=copy.deepcopy(i+1)
      
    else:
      p=copy.deepcopy(i+1)
      #print(p,i,'pi')
    
    
  elif (i in one):
    if j>0 and (len(NDSi0) == 0):
      i0=copy.deepcopy(X_train_image[p:(j+p)])
      l0=copy.deepcopy(y_train_label[p:(j+p)])
      NDSi0 = copy.deepcopy(i0)
      NDSl0 = copy.deepcopy(l0)
      j=copy.deepcopy(0)
      p=copy.deepcopy(i)
    elif j>0 and (len(NDSi0) > 0):
      i0=copy.deepcopy(X_train_image[p:(j+p)])
      l0=copy.deepcopy(y_train_label[p:(j+p)])
      NDSi0 = copy.deepcopy(np.concatenate((NDSi0, i0), axis=0))
      NDSl0 = copy.deepcopy(np.concatenate((NDSl0, l0), axis=0))
      j=copy.deepcopy(0)
      p=copy.deepcopy(i)
      #print(NDSi0.shape)
    else:
      None
    print(y_train_label[i],'[i]')
    print(i,'i')
    if ((len(NDSi1)+k) < 1454):
      k+=1
      print(k,'k')
      print(y_train_label[p],'[p]')
      print(p,'p')
    elif ((len(NDSi1)+k) == 1454) and k>0:
      i1=copy.deepcopy(X_train_image[p:(k+p)])
      l1=copy.deepcopy(y_train_label[p:(k+p)])
      NDSi1 = copy.deepcopy(np.concatenate((NDSi1, i1), axis=0))
      NDSl1 = copy.deepcopy(np.concatenate((NDSl1, l1), axis=0))
      k=copy.deepcopy(0)
      p=copy.deepcopy(i+1)
    else:
      p=copy.deepcopy(i+1)
    
  else:
    None
print(NDSi1.shape, NDSi0.shape)
print(NDSi1)
one = np.where(y_test_label == 1)
one = copy.deepcopy(one[0])
for i in range(len(X_test_image)):
  if (type(NDSi1) != int) and (i in one) and (len(NDSi1) < 1800):
    i1=copy.deepcopy(X_test_image[i])
    l1=copy.deepcopy(y_test_label[i])
    i1 = copy.deepcopy(i1[np.newaxis, :])
    l1 = copy.deepcopy(np.array([l1]))
    NDSi1 = copy.deepcopy(np.concatenate((NDSi1, i1), axis=0))
    NDSl1 = copy.deepcopy(np.concatenate((NDSl1, l1), axis=0))
NDSi=copy.deepcopy(np.concatenate((NDSi0, NDSi1), axis=0))
NDSl= copy.deepcopy(np.concatenate((NDSl0, NDSl1), axis=0))
print(NDSi.shape, NDSl.shape)
X_train, X_test, y_train, y_test = train_test_split(NDSi, NDSl, test_size=0.2, random_state=990, shuffle=True)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)
np.save('X_train', X_train)
np.save('X_test', X_test)
np.save('y_train', y_train)
np.save('y_test', y_test)

串流輸出內容已截斷至最後 5000 行。
1 [i]
299926 i
1 k
1 [p]
299926 p
[1] ytrain
(759, 20, 2, 3)
1 [i]
301891 i
1 k
1 [p]
301891 p
[1] ytrain
(760, 20, 2, 3)
1 [i]
302106 i
1 k
1 [p]
302106 p
[1] ytrain
(761, 20, 2, 3)
1 [i]
302515 i
1 k
1 [p]
302515 p
[1] ytrain
(762, 20, 2, 3)
1 [i]
302763 i
1 k
1 [p]
302763 p
[1] ytrain
(763, 20, 2, 3)
1 [i]
303268 i
1 k
1 [p]
303268 p
[1] ytrain
(764, 20, 2, 3)
1 [i]
303713 i
1 k
1 [p]
303713 p
[1] ytrain
(765, 20, 2, 3)
1 [i]
303753 i
1 k
1 [p]
303753 p
[1] ytrain
(766, 20, 2, 3)
1 [i]
303954 i
1 k
1 [p]
303954 p
[1] ytrain
(767, 20, 2, 3)
1 [i]
305467 i
1 k
1 [p]
305467 p
[1] ytrain
(768, 20, 2, 3)
1 [i]
305625 i
1 k
1 [p]
305625 p
[1] ytrain
(769, 20, 2, 3)
1 [i]
305713 i
1 k
1 [p]
305713 p
[1] ytrain
(770, 20, 2, 3)
1 [i]
306286 i
1 k
1 [p]
306286 p
[1] ytrain
(771, 20, 2, 3)
1 [i]
306486 i
1 k
1 [p]
306486 p
[1] ytrain
(772, 20, 2, 3)
1 [i]
306896 i
1 k
1 [p]
306896 p
[1] ytrain
(773, 20, 2, 3)
1 [i]
306936 i
1 k
1 [p]
306936 p
[1] ytrain
(774, 20, 2, 3)
1 [

In [36]:
print(NDSl1.shape)
one = np.where(NDSl1 == 1)
one = copy.deepcopy(one[0])
print(len(one))

(1800,)
1800


檢查

In [15]:
datafile1 = u'./X_train.npy'
datafile2 = u'./y_train.npy'
datafile3 = u'./X_test.npy'
datafile4 = u'./y_test.npy'
X_train_image,y_train_label,X_test_image,y_test_label = np.load(datafile1), np.load(datafile2), np.load(datafile3), np.load(datafile4)
#print(X_train_image[1:5],y_train_label[1:5])
one = np.where(y_train_label == 1)
one = copy.deepcopy(one[0])
print(len(one))

p = np.where(y_train_label == 0)
p = copy.deepcopy(p[0])
print(len(p))

on = np.where(y_test_label == 1)
on = copy.deepcopy(on[0])
print(on)

n = np.where(y_test_label == 0)
n = copy.deepcopy(n[0])
print(len(n))

472
79528
[  741   823  1044  1114  1303  1470  1918  2211  2601  2811  2898  3163
  3264  3578  3601  3877  4148  4858  5003  5174  5187  5879  6290  6609
  6848  7015  7275  7377  7687  7736  7866  8027  8178  8284  8446  8535
  8758  9054  9090  9230  9366  9669  9737  9744  9835 10025 10159 10167
 10172 10312 10372 10555 10819 10971 11083 11086 11169 11276 11287 11589
 11710 11742 11864 12132 12183 12382 12508 12641 12994 13071 13120 13292
 13341 13349 13389 13503 13566 13590 13988 14303 14326 14347 14506 14921
 14938 14971 15117 15285 15295 15508 15570 15575 15584 16036 16140 16765
 16879 17256 17854 17905 17940 17978 18071 18205 18243 18978 19141 19477
 19530 19665 19691 19984 19991]
19887


下載

In [ ]:
from google.colab import files
files.download("X_train.npy")